# Econ 8210 Homework 1, Fall 2024
### Numerical Integration and Optimization
### Haosi Shen

In [1]:
# Housekeeping
import numpy as np
import pandas as pd
import time

np.random.seed(42) 

## 1. Integration

Compute
\begin{equation*}
\int_{0}^{T} e^{-\rho t} u(1-e^{-\lambda t})\,dt
\end{equation*}
for $T=100$, $\rho = 0.04$, $\lambda = 0.02$, and $u(c)=-e^{-c}$ using **quadrature** (midpoint, Trapezoid, and Simpson rule) and Monte Carlo integration.

In [2]:
# Define Problem
T = 100
rho = 0.04
lambda_ = 0.02

def u(c):
    return -np.exp(-c)

def integrand(t):
    return np.exp(-rho * t) * u(1 - np.exp(-lambda_ * t))

# Number of intervals/draws
n_intervals = np.array([10, 100, 1000, 10000, 100000])

### Quadrature Integration

In [3]:
# Midpoint
def midpoint_quadrature(a, b, n):
    start_time = time.time() 

    h = (b - a) / n
    total = 0
    for i in range(n):
        midpoint = a + (i + 0.5) * h
        total += integrand(midpoint)

    integral_est = h * total
    end_time = time.time()
    comp_time = end_time - start_time # record computation time
    return integral_est, comp_time

vec_midpoint =  np.vectorize(midpoint_quadrature)
results_midpoint, times_midpoint = vec_midpoint(0, T, n_intervals)

In [4]:
# Trapezoid
def trapezoid_quadrature(a, b, n):
    start_time = time.time() 
    
    h = (b - a) / n
    total = 0.5 * (integrand(a) + integrand(b))
    for i in range(1, n):
        total += integrand(a + i * h)
    
    integral_est = h * total
    end_time = time.time()
    comp_time = end_time - start_time # record computation time
    return integral_est, comp_time

vec_trapezoid =  np.vectorize(trapezoid_quadrature)
results_trapezoid, times_trapezoid = vec_trapezoid(0, T, n_intervals)

In [5]:
# Simpson's Rule
def simpsons_quadrature(a, b, n):
    start_time = time.time() 
    
    if n % 2 == 1:
        n += 1  # ensure n is even
    h = (b - a) / n
    total = integrand(a) + integrand(b)
    for i in range(1, n, 2):
        total += 4 * integrand(a + i * h)
    for i in range(2, n, 2):
        total += 2 * integrand(a + i * h)
    
    integral_est = (h / 3) * total
    end_time = time.time()
    comp_time = end_time - start_time # record computation time
    return integral_est, comp_time

vec_simpsons =  np.vectorize(simpsons_quadrature)
results_simpsons, times_simpsons = vec_simpsons(0, T, n_intervals)

### Monte Carlo Integration

In [6]:
def monteCarlo_integration(a, b, n):
    start_time = time.time() 
    
    random_points = np.random.uniform(a, b, n)
    integral_est = (b - a) * np.mean([integrand(t) 
                                           for t in random_points])
    end_time = time.time()
    comp_time = end_time - start_time # record computation time
    return integral_est, comp_time

vec_monteCarlo =  np.vectorize(monteCarlo_integration)
results_monteCarlo, times_monteCarlo = vec_monteCarlo(0, T, n_intervals)

### Results

In [7]:
results_integration = pd.DataFrame(np.stack((results_midpoint, results_trapezoid, 
                                             results_simpsons, results_monteCarlo)),
            columns = ['N = 10', 'N = 100', 'N = 1000', 'N = 5000', 'N = 10000'], 
            index = (['Midpoint', 'Trapezoid', 'Simpson\'s', 'Monte Carlo']))


print("Integral Estimates")
display(results_integration)

Integral Estimates


,N = 10,N = 100,N = 1000,N = 5000,N = 10000
Midpoint,-17.964420,-18.207039,-18.209501,-18.209525,-18.209525
Trapezoid,-18.702748,-18.214498,-18.209575,-18.209526,-18.209525
Simpson's,-18.224641,-18.209527,-18.209525,-18.209525,-18.209525
Monte Carlo,-24.732456,-20.260672,-18.809211,-18.360223,-18.198811


In [8]:
times_integration = pd.DataFrame(np.stack((times_midpoint, times_trapezoid, 
                                             times_simpsons, times_monteCarlo)),
            columns = ['N = 10', 'N = 100', 'N = 1000', 'N = 5000', 'N = 10000'], 
            index = (['Midpoint', 'Trapezoid', 'Simpson\'s', 'Monte Carlo']))

print("Computation Time (seconds)")
display(times_integration)

Computation Time (seconds)


,N = 10,N = 100,N = 1000,N = 5000,N = 10000
Midpoint,0.000016,0.000137,0.001410,0.014347,0.146615
Trapezoid,0.000020,0.000166,0.001446,0.014592,0.189524
Simpson's,0.000019,0.000142,0.001401,0.014641,0.143357
Monte Carlo,0.000032,0.000156,0.001482,0.015456,0.139798


In alignment with the theoretical properties of each method,
> * The quadrature methods provide accurate results as the number of intervals $N$ increases, with Simpson's rule converging the fastest.
> * Monte Carlo integration has more variability but still trends toward the true value with higher number of draws $N$.
> * Regarding computation time, Midpoint and Simpson’s methods are generally faster and more efficient. Monte Carlo integration becomes competitive at larger $N$, while the trapezoid rule is generally slower.

In general, quadrature methods are faster and more accurate for lower-dimension problems and smaller $N$, whereas Monte Carlo becomes more competitive at large $N$ and higher dimensions. 

## 2. Optimization
